In [1]:
import numpy as np 
import joblib
import pandas as pd 
import os 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import StandardScaler

In [2]:
color_data= pd.read_csv('colors.csv') 
color_data.head()
X = color_data[['B','G','R']].values
y = color_data['color_name'].values
# Normalisasi data
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X)
# Split dataset untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [3]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [4]:
# Evaluasi model
train_acc = accuracy_score(y_train, knn.predict(X_train))
test_acc = accuracy_score(y_test, knn.predict(X_test))

print(f"Akurasi pada data latih: {train_acc * 100:.2f}%")


Akurasi pada data latih: 89.31%


In [5]:
# Simpan model
joblib.dump(knn, 'knn_model.pkl')
print("Model berhasil disimpan!")
# Buat ulang dan simpan scaler
scaler = StandardScaler()
scaler.fit(X)  # Pastikan X adalah dataset yang digunakan sebelumnya
joblib.dump(scaler, 'scaler.pkl')

print("Scaler berhasil disimpan!")

Model berhasil disimpan!
Scaler berhasil disimpan!


In [6]:
import cv2
import numpy as np 

# Muat model KNN dan scaler
knn = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Inisialisasi kamera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Ambil pixel tengah gambar
    height, width, _ = frame.shape
    pixel_center = frame[height // 2, width // 2]

    # Normalisasi sebelum prediksi
    pixel_center_scaled = scaler.transform([pixel_center])

    # Prediksi warna
    color_pred = knn.predict(pixel_center_scaled)[0]

    # Tampilkan warna pada frame
    cv2.putText(frame, f'Color: {color_pred}', (65, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 205, 50), 2)
    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
